# SF Crime Statistical Analysis

In [ ]:
# Import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Grab dataframe containing police reports
# Create df on Macbook
df = pd.read_csv("/Users/garrettyamane/Desktop/Springboard/capstone_project_1_data/merged.csv")

# Create df on PC
# df = pd.read_csv("/Users/Garrett/Desktop/Springboard/capstone_project_1_data/merged.csv")
df = df.drop(columns='Unnamed: 0')
df.head()